# Source Database Model Opdracht

Operationele Databases:

In [2]:
import sqlite3
import pandas as pd
import pyodbc

go_sales_conn = sqlite3.connect("../../../data/raw/go_sales_train.sqlite")
go_crm_conn = sqlite3.connect("../../../data/raw/go_crm_train.sqlite")
go_staff_conn = sqlite3.connect("../../../data/raw/go_staff_train.sqlite")

Connectie met SSMS voor SDM:

In [3]:
DB = {"servername": r"localhost,1433", "database": "sdm", "username": "sa", "password": "iDTyjZx7dRL4"}

export_conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={DB['servername']};"
    f"DATABASE={DB['database']};"
    f"UID={DB['username']};"
    f"PWD={DB['password']}"
)

OperationalError: ('HYT00', '[HYT00] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')

DataFrames maken voor tables:

In [ ]:
def create_dataframes_sql(connection, db_type):
    dictionary : dict = {}
    query : str = ""
    key : str = ""

    if (db_type == "sqlite"):
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif (db_type == "ssms"):
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"

    table_names = pd.read_sql(query, connection)

    for table in table_names[key].tolist():
        dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)

    return dictionary

go_sales_tables = create_dataframes_sql(go_sales_conn, "sqlite")
go_crm_tables = create_dataframes_sql(go_crm_conn, "sqlite")
go_staff_tables = create_dataframes_sql(go_staff_conn, "sqlite")

print(list(go_sales_tables.keys()))

SQLite Dictionaries mergen naar 1 Dictionary:

In [ ]:
go_sdm_tables = go_sales_tables | go_crm_tables | go_staff_tables # Alle drie mergen in 1 Dictionary met alle DataFrames

#Forecast en inventory tables toevoegen
inventory_df = pd.read_csv("../../../data/raw/inventory_levels_train.csv")
forecast = pd.DataFrame(columns=['PRODUCT_NUMBER', 'YEAR', 'MONTH', 'EXPECTED_VOLUME'])

go_sdm_tables["inventory_levels"] = inventory_df
go_sdm_tables["forecast"] = forecast

# DataFrames met ontbrekende/verkeerd genoemde rows updaten:
go_sdm_tables["country"]["LANGUAGE"] = go_sales_tables["country"]["LANGUAGE"]

go_sdm_tables["country"]["CURRENCY_NAME"] = go_sales_tables["country"]["CURRENCY_NAME"]

go_sdm_tables["country"] = go_sdm_tables["country"].rename(columns={'COUNTRY_EN': 'COUNTRY'})

go_sdm_tables["product_line"] = go_sales_tables["product_line"]

try:
    go_sdm_tables["retailer_headquarters"].drop('POSTAL_ZONE', axis=1, inplace=True)
except KeyError:
    print("Removal of Postal Zone in tables has already been complete.")

# Dictionary in goede volgorde zetten (om inserts goed te laten werken):
dict_order = [
    'sales_territory', 
    'country', 
    'order_method', 
    'retailer_site', 
    'sales_branch', 
    'sales_staff', 
    'retailer_contact', 
    'order_header', 
    'product_line', 
    'product_type', 
    'product', 
    'order_details', 
    'return_reason', 
    'returned_item', 
    'course', 
    'satisfaction_type', 
    'satisfaction', 
    'training',
    'age_group',
    'retailer_segment',
    'retailer_headquarters',
    'retailer_type',
    'retailer'
    'sales_demographic',
    'inventory_levels',
    'forecast'
]

go_sdm_tables = {k: go_sdm_tables[k] for k in dict_order if k in go_sdm_tables}

print(list(go_sdm_tables.keys()))


NameError: name 'go_sales_tables' is not defined

SSMS Databases vullen:

In [ ]:
export_cursor = export_conn.cursor()

for table_name, df in go_sdm_tables.items():
    try:
        for index, row in df.iterrows():
            columns = df.columns.tolist()

            values = []
            for col in columns:
                value = row[col]

                if pd.isna(value):
                    values.append("NULL")

                elif isinstance(value, str):
                    values.append(f"'{value.replace("'", "''")}'")

                else:
                    values.append(str(value))

            column_names = ", ".join(columns)
            value_string = ", ".join(values)
            query = f"INSERT INTO {table_name} ({column_names}) VALUES ({value_string})"

            export_cursor.execute(query)
    except pyodbc.Error as e:
        print(f"Error in table: {table_name}")
        print(f"Query: {query}")
        print(f"Error message: {e}")
        print("-" * 80)

export_conn.commit()
export_cursor.close()


Leegmaken van alle tables:

In [ ]:
export_cursor = export_conn.cursor()

tables = [
    "forecast",
    "inventory_levels",
    "sales_demographic",
    "retailer",
    "retailer_type",
    "retailer_headquarters",
    "retailer_segment",
    "age_group",
    "training",
    "satisfaction",
    "satisfaction_type",
    "course",
    "returned_item",
    "return_reason",
    "order_details",
    "product",
    "product_type",
    "product_line",
    "order_header",
    "retailer_contact",
    "sales_staff",
    "sales_branch",
    "retailer_site",
    "order_method",
    "country",
    "sales_territory"
]

for table in tables:
    query = f"DELETE FROM {table};"
    try:
        export_cursor.execute(query)
        print(f"Cleared table: {table}")
    except pyodbc.Error as e:
        print(f"Error clearing table {table}: {e}")

export_conn.commit()
export_cursor.close()